# Day11_2 별첨: 고급 감성 분석 (LLM & Embedding) - 정답

이 노트북은 Day11_2 별첨 퀴즈의 정답과 풀이를 담고 있습니다.

---

In [ ]:
# 필요한 라이브러리 임포트
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time

# Google Gemini API
try:
    from google import genai
    from google.genai import types
    GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
    client = genai.Client(api_key=GOOGLE_API_KEY) if GOOGLE_API_KEY else None
    gemini_available = True
except ImportError:
    client = None
    gemini_available = False

# sentence-transformers
try:
    from sentence_transformers import SentenceTransformer
    local_model = SentenceTransformer('jhgan/ko-sbert-nli')
    local_embedding_available = True
except ImportError:
    local_embedding_available = False

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

print("라이브러리 임포트 완료")

---

## Q1. Zero-shot 프롬프트 작성하기 ⭐

**문제**: 텍스트의 감성을 '긍정', '부정', '중립' 중 하나로 분류하는 Zero-shot 프롬프트를 작성하세요.

In [ ]:
# ✅ 정답
text = "이 영화 정말 재미없어요. 시간 낭비였어요."

prompt = f"""다음 텍스트의 감성을 '긍정', '부정', '중립' 중 하나로만 답하세요.

텍스트: {text}

감성:"""

print("작성된 프롬프트:")
print(prompt)

In [ ]:
# 테스트 (API가 있는 경우)
if client:
    response = client.models.generate_content(
        model="gemini-2.0-flash-exp",
        contents=prompt
    )
    print(f"LLM 응답: {response.text.strip()}")
else:
    print("기대 응답: 부정")

### 💡 Q1 풀이 설명

**접근 방법**:
- Zero-shot은 예제 없이 바로 분류 요청
- 응답 형식을 명확히 제한 ("하나로만 답하세요")

**핵심 개념**:
- 프롬프트는 명확한 지시와 제한이 중요
- 응답 형식을 지정하면 파싱이 쉬움

**대안**:
```python
# JSON 형식 요청
prompt = f'''텍스트: {text}
감성을 JSON으로 답하세요: {{"sentiment": "긍정/부정/중립"}}'''
```

---

## Q2. Few-shot 예제 구성하기 ⭐⭐

**문제**: 음식점 리뷰 감성 분류를 위한 Few-shot 예제를 5개 작성하세요.

In [ ]:
# ✅ 정답
food_examples = [
    ("국물이 진하고 면발이 쫄깃해요. 맛있어요!", "긍정"),
    ("양도 많고 가성비 최고입니다. 재방문 의사 100%!", "긍정"),
    ("너무 짜고 느끼해요. 위생도 별로인 것 같아요.", "부정"),
    ("주문한 지 1시간이 지나도 안 나와요. 최악!", "부정"),
    ("평범해요. 특별히 맛있지도 맛없지도 않아요.", "중립")
]

print("음식점 리뷰 Few-shot 예제:")
for text, sentiment in food_examples:
    print(f"  [{sentiment}] {text}")

In [ ]:
# Few-shot 프롬프트 구성
def create_few_shot_prompt(examples, target_text):
    prompt = "다음은 음식점 리뷰의 감성 분류 예시입니다.\n\n"
    
    for ex_text, ex_sentiment in examples:
        prompt += f"리뷰: {ex_text}\n감성: {ex_sentiment}\n\n"
    
    prompt += f"위 예시를 참고하여 다음 리뷰의 감성을 분류해주세요.\n\n"
    prompt += f"리뷰: {target_text}\n감성:"
    
    return prompt

# 테스트
test_review = "삼겹살이 신선하고 불맛이 좋아요!"
prompt = create_few_shot_prompt(food_examples, test_review)
print(prompt)

### 💡 Q2 풀이 설명

**접근 방법**:
- 균형 잡힌 예제 (긍정 2, 부정 2, 중립 1)
- 도메인 특화 표현 사용 (국물, 면발, 가성비 등)

**핵심 개념**:
- Few-shot 예제는 다양한 패턴을 포함해야 함
- 실제 리뷰 스타일과 유사하게 작성

**실무 팁**:
- 경계 케이스 예제 포함 (복합 감성 등)
- 도메인 용어 포함 (맛, 서비스, 위생 등)

---

## Q3. 코사인 유사도 계산하기 ⭐⭐

**문제**: 두 벡터 A와 B의 코사인 유사도를 직접 계산하는 함수를 작성하세요.

In [ ]:
# ✅ 정답
def cosine_similarity_manual(A, B):
    """
    코사인 유사도 수동 계산
    
    cos(A, B) = (A · B) / (||A|| × ||B||)
    """
    # 내적 (dot product)
    dot_product = np.dot(A, B)
    
    # 노름 (벡터 크기)
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)
    
    # 코사인 유사도
    similarity = dot_product / (norm_A * norm_B)
    
    return similarity


# 테스트
A = np.array([1, 2, 3])
B = np.array([4, 5, 6])

manual_result = cosine_similarity_manual(A, B)
sklearn_result = cosine_similarity([A], [B])[0][0]

print(f"A = {A}")
print(f"B = {B}")
print(f"\n수동 계산 결과: {manual_result:.6f}")
print(f"sklearn 결과: {sklearn_result:.6f}")
print(f"\n결과 일치: {np.isclose(manual_result, sklearn_result)}")

### 💡 Q3 풀이 설명

**접근 방법**:
- 코사인 유사도 공식 그대로 구현
- numpy 함수 활용 (dot, linalg.norm)

**핵심 개념**:
- 내적: `A · B = a1*b1 + a2*b2 + ...`
- 노름: `||A|| = sqrt(a1² + a2² + ...)`
- 코사인 유사도: 두 벡터 사이 각도의 코사인 값

**대안**:
```python
# numpy만 사용
def cosine_sim(A, B):
    return np.dot(A, B) / (np.sqrt(np.sum(A**2)) * np.sqrt(np.sum(B**2)))
```

---

## Q4. 5점 척도 감성 분류 프롬프트 ⭐⭐⭐

**문제**: LLM이 1~5점으로 감성을 평가하고 이유도 설명하는 프롬프트를 작성하세요.

In [ ]:
# ✅ 정답
text = "배송은 빨랐는데 제품 품질이 기대에 못 미쳐요."

prompt = f"""다음 고객 리뷰의 감성을 1~5점으로 평가해주세요.

평가 기준:
1점: 매우 부정 (불만, 환불 요청 수준)
2점: 부정 (실망, 아쉬움)
3점: 중립 (보통, 무난함)
4점: 긍정 (만족, 좋음)
5점: 매우 긍정 (감동, 최고)

리뷰: {text}

다음 형식으로 답변해주세요:
점수: [1-5 사이 숫자]
이유: [한 문장으로 평가 근거]"""

print("작성된 프롬프트:")
print(prompt)

In [ ]:
# 테스트
if client:
    response = client.models.generate_content(
        model="gemini-2.0-flash-exp",
        contents=prompt
    )
    print(f"LLM 응답:\n{response.text.strip()}")
else:
    print("기대 응답:")
    print("점수: 3")
    print("이유: 배송은 긍정적이나 품질 불만족으로 중립 수준")

### 💡 Q4 풀이 설명

**접근 방법**:
- 명확한 평가 기준 제시 (1~5점 각각 정의)
- 출력 형식 지정으로 파싱 용이

**핵심 개념**:
- 프롬프트에 평가 기준을 명시하면 일관성 향상
- 이유를 함께 요청하면 해석 가능성 확보

**실무 팁**:
- 도메인에 맞는 평가 기준 정의
- JSON 형식으로 요청하면 파싱이 더 쉬움

---

## Q5. 측면(Aspect) 기반 감성 분석 프롬프트 ⭐⭐⭐

**문제**: 리뷰에서 여러 측면별로 감성을 추출하는 프롬프트를 작성하세요.

In [ ]:
# ✅ 정답
text = "배송은 빠르고 좋았어요. 근데 품질이 별로고 가격도 비싸네요."

prompt = f"""다음 고객 리뷰에서 언급된 각 측면(aspect)별로 감성을 분석해주세요.

분석 가능한 측면: 배송, 품질, 가격, 서비스, 디자인, 기능 등

리뷰: {text}

다음 형식으로 답변해주세요 (언급된 측면만):
- [측면]: [긍정/부정/중립]

마지막에 전체 감성도 판단해주세요:
전체 감성: [긍정/부정/중립]"""

print("작성된 프롬프트:")
print(prompt)

In [ ]:
# 테스트
if client:
    response = client.models.generate_content(
        model="gemini-2.0-flash-exp",
        contents=prompt
    )
    print(f"LLM 응답:\n{response.text.strip()}")
else:
    print("기대 응답:")
    print("- 배송: 긍정")
    print("- 품질: 부정")
    print("- 가격: 부정")
    print("전체 감성: 부정")

### 💡 Q5 풀이 설명

**접근 방법**:
- 측면 목록 제시로 분석 범위 가이드
- 측면별 감성 + 전체 감성 요청

**핵심 개념**:
- ABSA (Aspect-Based Sentiment Analysis)
- 단순 긍정/부정보다 상세한 인사이트

**실무 활용**:
- 제품 개선점 파악 (어떤 측면이 부정인지)
- 경쟁사 비교 (측면별 비교 가능)

---

## Q6. 배치 감성 분석 함수 개선 ⭐⭐⭐

**문제**: 진행률 표시와 에러 처리를 추가한 배치 분석 함수를 작성하세요.

In [ ]:
# ✅ 정답
def analyze_batch_improved(texts, max_retries=3, delay=1):
    """
    개선된 배치 감성 분석 함수
    
    Parameters:
    - texts: 텍스트 리스트
    - max_retries: 최대 재시도 횟수
    - delay: API 호출 간 대기 시간
    
    Returns:
    - DataFrame: 분석 결과
    """
    results = []
    total = len(texts)
    
    for i, text in enumerate(texts):
        # 진행률 표시
        print(f"진행 중: {i+1}/{total} ({(i+1)/total*100:.1f}%)", end="\r")
        
        sentiment = None
        
        # 재시도 로직
        for attempt in range(max_retries):
            try:
                if client:
                    prompt = f"""다음 텍스트의 감성을 '긍정', '부정', '중립' 중 하나로만 답하세요.
텍스트: {text}
감성:"""
                    response = client.models.generate_content(
                        model="gemini-2.0-flash-exp",
                        contents=prompt
                    )
                    sentiment = response.text.strip()
                    break  # 성공하면 루프 종료
                else:
                    sentiment = "API 미설정"
                    break
            except Exception as e:
                print(f"\n  {i+1}번 항목 재시도 {attempt+1}/{max_retries}: {str(e)[:30]}")
                if attempt < max_retries - 1:
                    time.sleep(delay * 2)  # 재시도 시 대기 시간 늘림
        
        # 모든 재시도 실패 시
        if sentiment is None:
            sentiment = "error"
        
        results.append({
            "text": text,
            "sentiment": sentiment
        })
        
        # 다음 호출 전 대기
        if i < total - 1:
            time.sleep(delay)
    
    print(f"\n완료! {total}개 분석됨")
    
    # 에러 통계
    errors = sum(1 for r in results if r['sentiment'] == 'error')
    if errors > 0:
        print(f"  (에러: {errors}개)")
    
    return pd.DataFrame(results)


# 테스트
test_texts = [
    "좋아요!",
    "별로예요.",
    "그냥 그래요."
]

results = analyze_batch_improved(test_texts)
print("\n결과:")
print(results)

### 💡 Q6 풀이 설명

**접근 방법**:
- `\r`로 같은 줄에 진행률 업데이트
- try-except로 에러 처리
- for 루프로 재시도 구현

**핵심 개념**:
- API 호출은 항상 실패 가능성 고려
- 재시도 시 대기 시간 늘림 (exponential backoff)

**실무 팁**:
- 대량 처리 시 중간 결과 저장
- 로깅 추가로 디버깅 용이

---

## Q7. 임베딩 기반 감성 강도 측정 ⭐⭐⭐⭐

**문제**: 코사인 유사도로 -1 ~ +1 연속 감성 점수를 계산하세요.

In [ ]:
# ✅ 정답

# 임베딩 함수 선택
if local_embedding_available:
    def get_embedding(text):
        return local_model.encode(text)
    print("로컬 임베딩 사용")
else:
    def get_embedding(text):
        return np.random.randn(768)  # 시뮬레이션
    print("시뮬레이션 임베딩 사용")


def calculate_sentiment_score(text, pos_ref_emb, neg_ref_emb):
    """
    임베딩 기반 감성 점수 계산 (-1 ~ +1)
    
    Parameters:
    - text: 분석할 텍스트
    - pos_ref_emb: 긍정 참조 임베딩
    - neg_ref_emb: 부정 참조 임베딩
    
    Returns:
    - float: 감성 점수 (-1: 매우 부정, +1: 매우 긍정)
    """
    # 대상 텍스트 임베딩
    text_emb = get_embedding(text).reshape(1, -1)
    
    # 긍정/부정 유사도 계산
    pos_sim = cosine_similarity(text_emb, pos_ref_emb.reshape(1, -1))[0][0]
    neg_sim = cosine_similarity(text_emb, neg_ref_emb.reshape(1, -1))[0][0]
    
    # 감성 점수: 긍정 유사도 - 부정 유사도
    score = pos_sim - neg_sim
    
    # -1 ~ +1 범위로 클리핑 (선택사항)
    score = max(-1, min(1, score))
    
    return score, pos_sim, neg_sim

In [ ]:
# 참조 임베딩 생성
positive_texts = ["정말 좋아요!", "최고예요!", "만족합니다!"]
negative_texts = ["별로예요.", "실망입니다.", "최악이에요."]

# 평균 임베딩
pos_embeddings = np.array([get_embedding(t) for t in positive_texts])
neg_embeddings = np.array([get_embedding(t) for t in negative_texts])

pos_ref = np.mean(pos_embeddings, axis=0)
neg_ref = np.mean(neg_embeddings, axis=0)

print("참조 임베딩 생성 완료")

In [ ]:
# 테스트
test_reviews = [
    "완전 대박이에요! 최고!",
    "괜찮아요. 나쁘지 않아요.",
    "별로예요. 기대 이하입니다.",
    "최악이에요. 돈 아까워요."
]

print("감성 점수 계산 결과")
print("=" * 60)

for review in test_reviews:
    score, pos_sim, neg_sim = calculate_sentiment_score(review, pos_ref, neg_ref)
    print(f"\n리뷰: {review}")
    print(f"  긍정 유사도: {pos_sim:.3f}")
    print(f"  부정 유사도: {neg_sim:.3f}")
    print(f"  감성 점수: {score:.3f}")

### 💡 Q7 풀이 설명

**접근 방법**:
- 긍정/부정 참조 임베딩 생성
- 대상 텍스트와 각각 유사도 계산
- 차이값으로 연속 점수 산출

**핵심 개념**:
- 임베딩 공간에서 긍정/부정 방향 정의
- 상대적 유사도로 중간값도 표현 가능

**실무 팁**:
- 참조 텍스트 다양하게 선정
- 도메인별 참조 세트 구성

---

## Q8. LLM vs 임베딩 비교 분석 ⭐⭐⭐⭐

**문제**: 두 방식의 결과를 비교 시각화하세요.

In [ ]:
# ✅ 정답
comparison_reviews = [
    "완전 만족해요! 최고의 구매였어요.",
    "돈 아까워요. 실망입니다.",
    "그냥 그래요. 보통이에요.",
    "배송은 빨랐는데 품질이...",
    "가성비 좋아요! 추천합니다.",
    "기대했는데 별로네요.",
    "나쁘지 않아요.",
    "최악이에요. 환불했어요.",
    "디자인 예쁘고 기능도 좋아요.",
    "가격만 좀 저렴하면 좋겠어요."
]

# 시뮬레이션 결과 (API 없을 때)
llm_results = ["긍정", "부정", "중립", "부정", "긍정", "부정", "중립", "부정", "긍정", "중립"]
emb_results = ["긍정", "부정", "중립", "중립", "긍정", "부정", "긍정", "부정", "긍정", "부정"]

comparison_df = pd.DataFrame({
    'review': comparison_reviews,
    'LLM': llm_results,
    'Embedding': emb_results
})

# 일치 여부
comparison_df['일치'] = comparison_df['LLM'] == comparison_df['Embedding']

print("비교 결과:")
print(comparison_df)

In [ ]:
# 일치율 계산
agreement_rate = comparison_df['일치'].mean() * 100
print(f"\n일치율: {agreement_rate:.1f}%")

# 불일치 항목
disagreements = comparison_df[~comparison_df['일치']]
print(f"\n불일치 항목 ({len(disagreements)}개):")
for _, row in disagreements.iterrows():
    print(f"  리뷰: {row['review'][:30]}...")
    print(f"    LLM: {row['LLM']}, 임베딩: {row['Embedding']}")

In [ ]:
# 시각화
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('LLM 분류 분포', '임베딩 분류 분포')
)

colors = {'긍정': '#2ecc71', '부정': '#e74c3c', '중립': '#95a5a6'}

# LLM 분포
llm_counts = comparison_df['LLM'].value_counts()
fig.add_trace(
    go.Bar(
        x=llm_counts.index,
        y=llm_counts.values,
        marker_color=[colors.get(s, '#333') for s in llm_counts.index],
        name='LLM'
    ),
    row=1, col=1
)

# 임베딩 분포
emb_counts = comparison_df['Embedding'].value_counts()
fig.add_trace(
    go.Bar(
        x=emb_counts.index,
        y=emb_counts.values,
        marker_color=[colors.get(s, '#333') for s in emb_counts.index],
        name='임베딩'
    ),
    row=1, col=2
)

fig.update_layout(
    title=f'LLM vs 임베딩 분류 비교 (일치율: {agreement_rate:.1f}%)',
    showlegend=False,
    width=800,
    height=400
)

fig.show()

### 💡 Q8 풀이 설명

**접근 방법**:
- 동일 데이터에 두 방식 적용
- 일치/불일치 분석
- 시각화로 패턴 비교

**핵심 개념**:
- 방식 간 일치율 = 신뢰도 지표
- 불일치 케이스 = 경계 사례 파악

**실무 팁**:
- 불일치 케이스 수동 검토
- 앙상블로 성능 향상 가능

---

## Q9. 종합 감성 분석 파이프라인 ⭐⭐⭐⭐⭐

**문제**: LLM + 임베딩 앙상블 분류기를 구현하세요.

In [ ]:
# ✅ 정답
class AdvancedSentimentAnalyzer:
    """
    종합 감성 분석기
    - LLM (Zero-shot/Few-shot)
    - 임베딩 기반
    - 앙상블
    """
    
    def __init__(self, embedding_func=None):
        self.embedding_func = embedding_func or get_embedding
        self.pos_ref = None
        self.neg_ref = None
        self.neu_ref = None
    
    def fit(self, positive_texts, negative_texts, neutral_texts=None):
        """참조 임베딩 생성"""
        pos_embs = np.array([self.embedding_func(t) for t in positive_texts])
        neg_embs = np.array([self.embedding_func(t) for t in negative_texts])
        
        self.pos_ref = np.mean(pos_embs, axis=0)
        self.neg_ref = np.mean(neg_embs, axis=0)
        
        if neutral_texts:
            neu_embs = np.array([self.embedding_func(t) for t in neutral_texts])
            self.neu_ref = np.mean(neu_embs, axis=0)
        
        print("참조 임베딩 생성 완료")
    
    def _analyze_llm(self, text):
        """LLM 분석"""
        if not client:
            return None
        
        prompt = f"""다음 텍스트의 감성을 '긍정', '부정', '중립' 중 하나로만 답하세요.
텍스트: {text}
감성:"""
        try:
            response = client.models.generate_content(
                model="gemini-2.0-flash-exp",
                contents=prompt
            )
            return response.text.strip()
        except:
            return None
    
    def _analyze_embedding(self, text):
        """임베딩 분석"""
        if self.pos_ref is None:
            return None, {}
        
        text_emb = self.embedding_func(text).reshape(1, -1)
        
        sims = {
            '긍정': cosine_similarity(text_emb, self.pos_ref.reshape(1, -1))[0][0],
            '부정': cosine_similarity(text_emb, self.neg_ref.reshape(1, -1))[0][0]
        }
        
        if self.neu_ref is not None:
            sims['중립'] = cosine_similarity(text_emb, self.neu_ref.reshape(1, -1))[0][0]
        
        prediction = max(sims, key=sims.get)
        return prediction, sims
    
    def analyze(self, text, method='ensemble'):
        """
        텍스트 감성 분석
        
        Parameters:
        - text: 분석할 텍스트
        - method: 'llm', 'embedding', 'ensemble'
        
        Returns:
        - dict: 분석 결과
        """
        result = {'text': text, 'method': method}
        
        if method == 'llm':
            result['prediction'] = self._analyze_llm(text)
            result['confidence'] = 'high' if result['prediction'] else 'error'
        
        elif method == 'embedding':
            pred, sims = self._analyze_embedding(text)
            result['prediction'] = pred
            result['similarities'] = sims
            if sims:
                max_sim = max(sims.values())
                result['confidence'] = 'high' if max_sim > 0.7 else 'medium' if max_sim > 0.5 else 'low'
        
        elif method == 'ensemble':
            llm_pred = self._analyze_llm(text)
            emb_pred, sims = self._analyze_embedding(text)
            
            result['llm_prediction'] = llm_pred
            result['embedding_prediction'] = emb_pred
            result['similarities'] = sims
            
            # 앙상블: 일치하면 해당 값, 불일치하면 임베딩 우선
            if llm_pred == emb_pred:
                result['prediction'] = llm_pred
                result['confidence'] = 'high'
            else:
                result['prediction'] = emb_pred or llm_pred
                result['confidence'] = 'medium'
        
        return result
    
    def analyze_batch(self, texts, method='ensemble'):
        """배치 분석"""
        results = []
        for i, text in enumerate(texts):
            print(f"진행: {i+1}/{len(texts)}", end="\r")
            result = self.analyze(text, method)
            results.append(result)
            if client and method in ['llm', 'ensemble']:
                time.sleep(1)
        print(f"\n완료: {len(texts)}개")
        return results

In [ ]:
# 테스트
analyzer = AdvancedSentimentAnalyzer()

# 학습
analyzer.fit(
    positive_texts=["좋아요!", "최고예요!", "만족합니다!"],
    negative_texts=["별로예요.", "실망입니다.", "최악이에요."],
    neutral_texts=["그냥 그래요.", "보통이에요.", "무난해요."]
)

# 분석
test_text = "가격은 비싸지만 품질이 좋아요."

print("\n임베딩 분석:")
print(analyzer.analyze(test_text, method='embedding'))

if client:
    print("\nLLM 분석:")
    print(analyzer.analyze(test_text, method='llm'))
    
    print("\n앙상블 분석:")
    print(analyzer.analyze(test_text, method='ensemble'))

### 💡 Q9 풀이 설명

**접근 방법**:
- 클래스로 상태 관리 (참조 임베딩)
- 메서드별 분석 로직 분리
- 앙상블: 일치=high, 불일치=medium

**핵심 개념**:
- 앙상블: 여러 모델 결과 종합
- 신뢰도: 일치 여부로 판단

**실무 팁**:
- 불일치 시 사람 검토 플래그
- 도메인별 가중치 조정

---

## Q10. 감성 분석 대시보드 ⭐⭐⭐⭐⭐

**문제**: 종합 대시보드를 만드세요.

In [ ]:
# ✅ 정답
dashboard_reviews = [
    "정말 좋아요! 만족합니다.",
    "별로예요. 실망입니다.",
    "그냥 그래요.",
    "최고의 선택이었어요!",
    "돈 아까워요.",
    "가격 대비 괜찮아요.",
    "다시는 안 살 거예요.",
    "추천합니다!",
    "보통이에요.",
    "완전 실망이에요.",
    "기대 이상이에요!",
    "그냥 무난해요."
]

# 시뮬레이션 결과
analysis_results = pd.DataFrame({
    'review': dashboard_reviews,
    'llm_pred': ['긍정', '부정', '중립', '긍정', '부정', '긍정', '부정', '긍정', '중립', '부정', '긍정', '중립'],
    'emb_pred': ['긍정', '부정', '중립', '긍정', '부정', '중립', '부정', '긍정', '중립', '부정', '긍정', '긍정'],
    'sentiment_score': [0.8, -0.7, 0.1, 0.9, -0.8, 0.3, -0.9, 0.7, 0.0, -0.85, 0.75, 0.15]
})

# 앙상블 결과
analysis_results['ensemble'] = analysis_results.apply(
    lambda row: row['llm_pred'] if row['llm_pred'] == row['emb_pred'] else row['emb_pred'],
    axis=1
)

analysis_results['agree'] = analysis_results['llm_pred'] == analysis_results['emb_pred']

print("분석 데이터:")
print(analysis_results)

In [ ]:
# 대시보드 생성
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{'type': 'pie'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'histogram'}]],
    subplot_titles=('전체 감성 분포', 'LLM vs 임베딩 비교',
                   '감성별 평균 점수', '감성 점수 분포')
)

colors = {'긍정': '#2ecc71', '부정': '#e74c3c', '중립': '#95a5a6'}

# 1. 전체 감성 분포 (파이)
ensemble_counts = analysis_results['ensemble'].value_counts()
fig.add_trace(
    go.Pie(
        labels=ensemble_counts.index,
        values=ensemble_counts.values,
        marker_colors=[colors.get(s, '#333') for s in ensemble_counts.index],
        hole=0.4
    ),
    row=1, col=1
)

# 2. LLM vs 임베딩 비교 (그룹 막대)
llm_counts = analysis_results['llm_pred'].value_counts()
emb_counts = analysis_results['emb_pred'].value_counts()

for sentiment in ['긍정', '부정', '중립']:
    fig.add_trace(
        go.Bar(
            x=['LLM', '임베딩'],
            y=[llm_counts.get(sentiment, 0), emb_counts.get(sentiment, 0)],
            name=sentiment,
            marker_color=colors[sentiment],
            showlegend=True
        ),
        row=1, col=2
    )

# 3. 감성별 평균 점수
sentiment_scores = analysis_results.groupby('ensemble')['sentiment_score'].mean()
fig.add_trace(
    go.Bar(
        x=sentiment_scores.index,
        y=sentiment_scores.values,
        marker_color=[colors.get(s, '#333') for s in sentiment_scores.index],
        showlegend=False
    ),
    row=2, col=1
)

# 4. 감성 점수 분포 (히스토그램)
fig.add_trace(
    go.Histogram(
        x=analysis_results['sentiment_score'],
        nbinsx=10,
        marker_color='#3498db',
        showlegend=False
    ),
    row=2, col=2
)

fig.update_layout(
    title='감성 분석 종합 대시보드',
    height=700,
    width=1000,
    barmode='group'
)

fig.show()

In [ ]:
# 인사이트 도출
print("=" * 60)
print("📊 인사이트")
print("=" * 60)

# 인사이트 1: 전체 감성 분포
total = len(analysis_results)
positive_rate = (analysis_results['ensemble'] == '긍정').sum() / total * 100
negative_rate = (analysis_results['ensemble'] == '부정').sum() / total * 100

print(f"\n1. 전체 감성 분포")
print(f"   - 긍정 비율: {positive_rate:.1f}%")
print(f"   - 부정 비율: {negative_rate:.1f}%")
print(f"   → {'긍정 우세' if positive_rate > negative_rate else '부정 우세'}")

# 인사이트 2: LLM vs 임베딩 일치율
agreement_rate = analysis_results['agree'].mean() * 100
print(f"\n2. 방법 간 일치율: {agreement_rate:.1f}%")
print(f"   → {'높은 신뢰도' if agreement_rate >= 80 else '검토 필요 케이스 있음'}")

# 인사이트 3: 평균 감성 점수
avg_score = analysis_results['sentiment_score'].mean()
print(f"\n3. 평균 감성 점수: {avg_score:.2f}")
print(f"   → {'긍정적 경향' if avg_score > 0 else '부정적 경향'}")

### 💡 Q10 풀이 설명

**접근 방법**:
- make_subplots로 2x2 레이아웃
- 다양한 차트 유형 조합
- 인사이트 자동 도출

**핵심 개념**:
- 대시보드: 여러 시각화 한 화면 구성
- 인사이트: 데이터에서 의미 추출

**실무 팁**:
- 핵심 지표 상단 배치
- 액션 가능한 인사이트 제공
- 시간 변화 추이 추가 고려

---

## 📊 학습 정리

### 퀴즈 핵심 요약

| 퀴즈 | 핵심 개념 | 난이도 |
|------|----------|--------|
| Q1 | Zero-shot 프롬프트 작성 | ⭐ |
| Q2 | Few-shot 예제 구성 | ⭐⭐ |
| Q3 | 코사인 유사도 수식 | ⭐⭐ |
| Q4 | 5점 척도 프롬프트 | ⭐⭐⭐ |
| Q5 | ABSA 프롬프트 | ⭐⭐⭐ |
| Q6 | 배치 처리 + 에러 핸들링 | ⭐⭐⭐ |
| Q7 | 임베딩 감성 점수 | ⭐⭐⭐⭐ |
| Q8 | LLM vs 임베딩 비교 | ⭐⭐⭐⭐ |
| Q9 | 앙상블 분류기 클래스 | ⭐⭐⭐⭐⭐ |
| Q10 | 종합 대시보드 | ⭐⭐⭐⭐⭐ |

### 💡 종합 팁

1. **프롬프트는 명확하게**: 출력 형식 지정 필수
2. **Few-shot은 균형 있게**: 각 클래스 예제 포함
3. **임베딩은 참조가 핵심**: 좋은 참조 = 좋은 분류
4. **앙상블로 신뢰도 향상**: 일치=high, 불일치=검토
5. **시각화로 검증**: 결과 분포 항상 확인